# Imports

In [1]:
import numpy as np
from glob import glob
import torch
import os, sys, time, csv
import re
import random
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as ipyw
import inspect
import pickle
%matplotlib inline

# Process image stacks into feature vector matrices using pre-trained model (for LSTM input).

In [2]:
# Torch imports.
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision.models.resnet

# Visualization imports
from IPython import display


In [3]:
# Part A: Load in model pre-trained on individual images. Note that pre-training may not even be necessary, as I was only
# changing the weights on the last (new) fully connected layer, and that layer will likely be gotten rid of. 
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

# Assemble image pathnames and understand class breakdown in dataset

In [4]:
# Note down train, validation and test directories.
topDir = 'Split_Data'
# topDir = 'Tiny_Data'

saveName = 'Testing'

train_dir = topDir + "/train"
valid_dir = topDir + "/valid"
test_dir = topDir + "/test"

# Check quantities of train, validation and test images
train_images = np.array(glob(train_dir + "/*/*"))
valid_images = np.array(glob(valid_dir + "/*/*"))
test_images = np.array(glob(test_dir + "/*/*"))

# Check relative percentages of image types
train_images_absent = np.array(glob(train_dir + "/Absent/*"))
train_images_present = np.array(glob(train_dir + "/Present/*"))

valid_images_absent = np.array(glob(valid_dir + "/Absent/*"))
valid_images_present = np.array(glob(valid_dir + "/Present/*"))

test_images_absent = np.array(glob(test_dir + "/Absent/*"))
test_images_present = np.array(glob(test_dir + "/Present/*"))

num_train_images = len(train_images)
num_valid_images = len(valid_images)
num_test_images = len(test_images)

print("There are {} training stacks, {} validation stacks and {} test stacks.".format(len(train_images),len(valid_images),len(test_images)))
print("For the training stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(train_images_present)/len(train_images)*100, pos2=len(train_images_present),neg=len(train_images_absent)/len(train_images)*100, neg2=len(train_images_absent)))
print("For the validation stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(valid_images_present)/len(valid_images)*100, pos2=len(valid_images_present),neg=len(valid_images_absent)/len(valid_images)*100, neg2=len(valid_images_absent)))
print("For the test stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(test_images_present)/len(test_images)*100, pos2=len(test_images_present),neg=len(test_images_absent)/len(test_images)*100, neg2=len(test_images_absent)))

There are 9123 training stacks, 1141 validation stacks and 1141 test stacks.
For the training stacks, 50.1% (4574) are positive and 49.9% (4549) are negative.
For the validation stacks, 50.1% (572) are positive and 49.9% (569) are negative.
For the test stacks, 50.1% (572) are positive and 49.9% (569) are negative.


In [5]:
# Check for GPU support
use_cuda = torch.cuda.is_available()

In [6]:
# Check to see how many GPUs are available.
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    num_devices = torch.cuda.device_count()
    print("{} GPUs found.".format(num_devices))
else:
    num_devices = 0

1 GPUs found.


# Plot sample image stacks

In [16]:
# Following https://github.com/mohakpatel/ImageSliceViewer3D, with modifications for plotting multiple stacks of color images. 
class ImageSliceViewer:
    """ 
    ImageSliceViewer is for viewing image stacks in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image.

    Argumentss:
    Volume = stack of color input images [num images, height, width, RGB]
    figsize = default(8,8), to set the size of the figure
    
    """
    
    def __init__(self, volume, figsize=(10,10)):
        self.volume = volume
        self.figsize = figsize
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # View the volume
        maxZ = self.volume.shape[0] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        fig=plt.figure(figsize=self.figsize)
        plt.imshow(self.volume[z,:,:,:], 
            vmin=self.v[0], vmax=self.v[1])
# ''';

In [19]:
def show_an_image_stack(dataset):
    fig = plt.figure(figsize = (10,10))
    axes = []
    # Grab a stack at random and build a volume.
    directories = os.listdir(dataset)
    directory = directories[np.random.randint(0,len(directories)-1)]
    files = os.listdir(os.path.join(dataset,directory))
    print("Plotting image stack {}".format(os.path.join(dataset,directory)))
    files.sort()
    imgArr = []
    for file in files:
        file = os.path.join(dataset,directory, file)
        file = pickle.load(file)
        imgArr.append(np.array(Image.open(file)))
    imgArr = np.array(imgArr)
    # Plot the volume within a subplot
    ImageSliceViewer(imgArr)
# ''';

In [21]:
# show_an_image_stack('Split_Data/train/Present')

# Imports for torch and DALI functions

In [7]:
# Define the model
class MM_DNN(nn.Module):
    def __init__(self, drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.lr = lr
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(5*1024,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,2)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        x = x.view(-1)
        out = self.dropout(self.relu(self.fc(x)))
        out = self.dropout(self.relu(self.fc2(out)))
        out = self.fc3(out)
        # return the final output
        return out
    


In [8]:
# Use the PyTorch data loader here, way easier. 
def load_pickle(path):
    with open(path,'rb') as f:
        return pickle.load(f)
    
train_data = datasets.DatasetFolder(train_dir,load_pickle,'.pickle')
validation_data = datasets.DatasetFolder(valid_dir,load_pickle,'.pickle')
test_data = datasets.DatasetFolder(test_dir,load_pickle,'.pickle')
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 1, shuffle=True, drop_last=True, num_workers=num_devices*4, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(validation_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True)

In [32]:
x = load_pickle('/home/ubuntu/src/YPB-AI/Data/7_Sample5_Dark_12-Mar-2019_11-41-58.pickle')
len(x[3][0])
image, label = iter(train_loader).next()
print(train_data)

Dataset DatasetFolder
    Number of datapoints: 9123
    Root Location: Split_Data/train
    Transforms (if any): None
    Target Transforms (if any): None


In [9]:
def train(net, train_loader, epochs=10, batch_size=2, lr=0.001, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: DNN network
        train_loader: PyTorch dataloader containing data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        lr: learning rate
        print_every: Number of steps for printing training and validation loss
    
    '''
    valid_batch = 1
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    running_loss = 0.
    
    
    if(use_cuda):
        net.cuda()
    all_valid_losses = [.6]
    counter = 0
    for e in range(epochs):
        
        for batch_idx, batch_data in enumerate(train_loader):
            inputs = batch_data[0].view(12,batch_size,1024)
            inputs = inputs[0:5] # Can try increase this to compare; original = 4 
            targets = batch_data[1]
            if(use_cuda):
                inputs, targets = inputs.cuda(), targets.cuda()
            # zero accumulated gradients
            net.zero_grad()
            # get the output from the model
            output = net(inputs)
            # calculate the loss and perform backprop
            loss = criterion(output.view(batch_size,2), targets)
            loss.backward()
            opt.step()
            running_loss += loss.item()
            
            # loss stats
            if batch_idx % print_every == 0:
                # Get validation loss
                val_losses = []
                train_losses = []
                net.eval()
                correct = 0.
                total = 0.
                for val_batch_idx, batch_data in enumerate(valid_loader):                    
                    inputs, targets = batch_data[0].view(12,valid_batch,1024), batch_data[1]
                    inputs = inputs[0:5]
                    
                    if(use_cuda):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output = net(inputs)
                    print(output)
                    
                    val_loss = criterion(output.view(valid_batch,2), targets)
                    val_losses.append(val_loss.item()/len(valid_loader))
                    
                    train_losses.append(running_loss/len(train_loader))
                    
                    pred = output.max(0,keepdim=True)[1]
                    print(pred)
                    
                    # compare predictions to true label
                    correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
                    total += valid_batch
                
                net.train() # reset to train mode after iteration through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(batch_idx),
                      "Loss: {:.8f}...".format(loss.item()),
                      "Val Loss: {:.4f}...".format(np.mean(val_losses)),
                      "Val Accuracy: {:.1f}%".format(correct/total*100))
                
                if np.mean(val_losses) < np.min(all_valid_losses):
                    print("Validation loss dropped, saving model.")
                    # torch.save(net,'DNNModel_' + saveName + '.pt')
                    all_valid_losses = np.append(all_valid_losses,np.mean(val_losses))

In [10]:
# define and print the net

net = MM_DNN()
#net = MM_LSTM(32,1)
print(net)

MM_DNN(
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=5120, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


In [11]:
train(net,train_loader,50,1,print_every=6000,lr=0.0001)

tensor([0.1487, 0.0412], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1487, 0.0439], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1428, 0.0516], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1522, 0.0458], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1492, 0.0408], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1387, 0.0562], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1099, 0.0429], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1650, 0.0459], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1414, 0.0485], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1437, 0.0453], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1404, 0.0496], device='cuda:0', grad_fn=

tensor([0.1567, 0.0475], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1473, 0.0391], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1565, 0.0435], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1568, 0.0424], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1575, 0.0544], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1695, 0.0408], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1660, 0.0528], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1547, 0.0333], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1535, 0.0391], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1542, 0.0373], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1668, 0.0577], device='cuda:0', grad_fn=

tensor([0.1324, 0.0431], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1413, 0.0450], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1368, 0.0394], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1053, 0.0246], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1449, 0.0452], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1299, 0.0445], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1239, 0.0391], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1431, 0.0483], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1195, 0.0266], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1460, 0.0597], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1147, 0.0312], device='cuda:0', grad_fn=

tensor([0.1287, 0.0387], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1258, 0.0361], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1263, 0.0366], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1245, 0.0306], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1161, 0.0191], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1268, 0.0372], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1322, 0.0362], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1283, 0.0382], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1283, 0.0380], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1292, 0.0374], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1279, 0.0391], device='cuda:0', grad_fn=

tensor([0], device='cuda:0')
tensor([0.1430, 0.0430], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1445, 0.0411], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1428, 0.0409], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1455, 0.0373], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1444, 0.0409], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1432, 0.0384], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1329, 0.0375], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1341, 0.0383], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1340, 0.0386], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1348, 0.0398], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1329, 0.040

...

tensor([0.1482, 0.0192], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1367, 0.0261], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1615, 0.0277], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([ 0.1626, -0.0051], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1833, 0.0869], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1605, 0.0261], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1185, 0.0281], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2166, 0.0355], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1322, 0.0371], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1263, 0.0057], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1128, 0.0184], device='cuda:0', grad_f

tensor([ 0.1002, -0.0188], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1305, 0.0395], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.0921, 0.0302], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1316, 0.0519], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1931, 0.0345], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([ 0.0942, -0.0006], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1372, 0.0227], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1772, 0.0035], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1461, 0.0445], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1503, 0.0488], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1525, 0.0475], device='cuda:0', grad

tensor([0.1453, 0.0349], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1454, 0.0272], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1842, 0.0075], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1186, 0.0333], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1561, 0.0285], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([ 0.1347, -0.0012], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([ 0.1393, -0.0084], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1421, 0.0164], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2039, 0.1106], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1718, 0.0311], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1925, 0.0062], device='cuda:0', grad

tensor([0.1924, 0.0525], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1955, 0.0707], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1811, 0.0580], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1898, 0.0132], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1924, 0.0335], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1568, 0.0825], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2292, 0.1045], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2032, 0.0387], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2126, 0.0435], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.2107, 0.0385], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0], device='cuda:0')
tensor([0.1979, 0.0373], device='cuda:0', grad_fn=

KeyboardInterrupt: 

In [ ]:
# Plot learning curve
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

In [13]:
net = torch.load('DNNModel_' + saveName + '.pt')
stateDictName ='DNN_StateDict_' + saveName + '.pt'

In [38]:
torch.save(net.state_dict(), stateDictName)

In [14]:
net.load_state_dict(torch.load(stateDictName))

In [29]:
# Test data
print(test_loader)

def test(net, test_loader):
    net.eval()
    correct = 0.
    total = 0.
    test_batch = 1
    test_losses = []
    criterion = nn.CrossEntropyLoss()
    for test_batch_idx, batch_data in enumerate(test_loader):                    
        inputs, targets = batch_data[0].view(12,test_batch,1024), batch_data[1]
        inputs = inputs[0:4]

        if(use_cuda):
            inputs, targets = inputs.cuda(), targets.cuda()
            
        # print(inputs)
        output = net(inputs)

        test_loss = criterion(output.view(test_batch,2), targets)
        test_losses.append(test_loss.item()/len(test_loader))

        pred = output.max(0,keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
        total += test_batch
        
    print("Test Loss: {:.8f}...".format(np.mean(test_losses)),
      "Test Accuracy: {:.1f}%".format(correct/total*100))
# plt.imshow(torchvision.utils.make_grid(images))

# outputs = net(images)

In [30]:
test(net, test_loader)

Test Loss: 0.00005668... Test Accuracy: 97.8%


In [40]:
net.load_state_dict(torch.load('DNN_StateDict_S3_Iter_1.pt'))

In [58]:
net.eval()
correct = 0.
total = 0.
false_pos = 0.
false_neg = 0.
test_batch = 1
test_losses = []
criterion = nn.CrossEntropyLoss()
for test_batch_idx, batch_data in enumerate(test_loader):                    
    inputs, targets = batch_data[0].view(12,test_batch,1024), batch_data[1]
    inputs = inputs[0:4]

    if(use_cuda):
        inputs, targets = inputs.cuda(), targets.cuda()

    # print(inputs)
    output = net(inputs)

    test_loss = criterion(output.view(test_batch,2), targets)
    test_losses.append(test_loss.item()/len(test_loader))

    pred = output.max(0,keepdim=True)[1]
    
    # compare predictions to true label
    correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
    
    # number of false positives; hack for batch size = 1
    false_pos += int(targets.data.cpu().numpy()[0] == 0 and pred.data.cpu().numpy()[0] == 1)
    
    # number of false negatives; hack for batch size = 1
    false_neg += int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)
    
    total += test_batch

In [59]:
print(correct)
print(total)
print(false_pos)
print(false_neg)
int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)

1116.0
1141.0
15.0
10.0


0

# Zip up state dict, get over to S3

In [90]:
# The below lines are only needed if allowing SageMaker to perform inference on the model.
# I can walk you through that when you get there.

#tarName = 'DNNModel_' + saveName + '.tar.gz' 

In [91]:
#os.system('tar -cvzf ' + tarName + ' ' + stateDictName)
#os.system('aws s3 cp ' + tarName + ' s3://ypb-ml-images/' + tarName)

0